In [ ]:
######Brian Snyder  gCOD Gust Wind Speed
######Revised by B Snyder 10/9/19.  Added to Watson Studio
# Gets multiple gCOD gCOD Gust Wind Speed layers from our featurizer catalog and saves them as individual layers in ESRI
# There are 5 layers that updated in this script using the 5 esriIds on ARCGIS Online below:
# esriIds = ["c2ff73fa17fb4d75ad0a0aa1a532631e","64ce16187717463c81716404284f4fc4","a5e6c77b59f8453884c470aea903457c", "b922d2367516450b8f6633546c04570c","34e3a9cc00a7433aa4277655e7cb2918"]
import requests,json,datetime # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(40) #  gCOD Gust Wind Speed 
prod_name='Windspeedgustsurface' # Per TWC naming conventions
APIKEY='THE_WEATHER_COMPANY_KEY_HERE'  #Enter your API key 


########################PRODUCT INVENTORY CALL################################
#We get the most recent runtime.
def InvCall():
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2
    print("Call #1: " + url)
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json() 
        for x in range(0,1):
            for y in range(0,1):
                timestamp = data["layers"][prod_num][prod_name]["dimensions"][x]["t"][y]
            return timestamp

##########################   API CALL   ###################################
def APIcall(timestamp,ts): # function for pulling data 
    count=0
    num_layers = 5  
    inc=10           
    low_limit = 20   ##Thresholds are in F.  Starting with negative
    low_limitS = str(low_limit)
    high_limit=low_limit+5
    high_limitS = str(high_limit)      
                 
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&t='+timestamp + '&apiKey='+APIKEY+'&geometryType=polygon'
    esriIds = ["c2ff73fa17fb4d75ad0a0aa1a532631e","64ce16187717463c81716404284f4fc4","a5e6c77b59f8453884c470aea903457c", "b922d2367516450b8f6633546c04570c","34e3a9cc00a7433aa4277655e7cb2918"]
    for x in range(0,num_layers):
        fc = {
            "type": "FeatureCollection",
            "features": []
        }
        count=count+1
        base3 = '&threshold='+low_limitS

        url = base1 + base2 + base3  # make API URL   
        print ("Call #2: " + url) 
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']            
            windGustSpeed= str(y['properties']['threshold'])
            f['properties']['Wind Gust Speed'] = windGustSpeed  
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
          
        
        #Update each piece of content in ESRI        
        with open("gCOD Gust Wind Speed_" + str(low_limitS) + ".geojson", "w") as outfile:
            json.dump(fc, outfile)
        if(len(fc['features']) != 0):
            portalURL = "http://ibmmain.maps.arcgis.com"
            userID = "ESRI_USERNAME"
            password = "ESRI_PASSWORD"
            gis = GIS(portalURL, userID, password)
            existing_ID = str(esriIds[x])  # Map ID
            wx_item = gis.content.get(existing_ID)
            collection = features.FeatureLayerCollection.fromitem(wx_item)
            collection.manager.overwrite('gCOD Gust Wind Speed_' + str(low_limitS) + '.geojson')  
            print ("Updating Layer: "+ str(low_limitS))
        else:
            print ("NOT Updating Layer: "+ str(low_limitS) + " due to no results for this layer")

        #reset limits
        low_limit=low_limit+inc
        low_limitS=str(low_limit)                  
        high_limit=high_limit+inc
        high_limitS=str(high_limit)

################################################################################
timestamp = InvCall()  ## Grab time stamp
print(timestamp)
fmt = "%Y-%m-%d %H:%M:%S"
# local time
#t = datetime.datetime.fromtimestamp(int(timeInfo[1])/1000)
#print (t.strftime(fmt)) # prints 2012-08-28 02:45:17
# utc time
t_utc = datetime.datetime.utcfromtimestamp(int(timestamp)/1000)
t_utc_formatted = t_utc.strftime(fmt)
APIcall(timestamp,t_utc_formatted)